In [1]:
import pandas as pd
import numpy as np

import json
import requests
from config import google_api_key

import warnings
warnings.filterwarnings('ignore')

In [2]:
test_data = pd.read_csv('Resources/test_data.csv')

In [3]:
test_data.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,UNIQUE KEY,VEHICLE TYPE CODE 1
0,2017-02-12,4:15,QUEENS,11106.0,40.758550,-73.924866,"40.75855, -73.924866",34 AVENUE,34 STREET,0.0,0.0,0,0,0,0,0,0,Unsafe Speed,3613857,SPORT UTILITY / STATION WAGON
1,2013-07-18,12:05,BROOKLYN,11209.0,40.617896,-74.029302,"40.6178962, -74.0293024",92 STREET,5 AVENUE,0.0,0.0,0,0,0,0,0,0,Unspecified,143196,PASSENGER VEHICLE
2,2015-09-16,9:00,MANHATTAN,10016.0,40.745682,-73.972125,"40.7456822, -73.9721247",EAST 37 STREET,1 AVENUE,0.0,0.0,0,0,0,0,0,0,Other Vehicular,3297366,SPORT UTILITY / STATION WAGON
3,2013-01-19,13:15,NaN,NaN,NaN,NaN,NaN,EAST 135 STREET,LINCOLN AVENUE,0.0,0.0,0,0,0,0,0,0,Other Vehicular,73713,PASSENGER VEHICLE
4,2018-08-11,22:51,BROOKLYN,11219.0,40.627350,-74.009964,"40.62735, -74.009964",NaN,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,3959547,Sedan


In [4]:
test_data.shape

(152, 20)

In [5]:
# Percent of zip codes missing before augmentation
percent_missing_from_test_data = round(test_data['ZIP CODE'].isna().sum()/len(test_data['ZIP CODE']) * 100, 2)
percent_missing_from_test_data

28.95

## 1. Reverse geocoding using Lat/Lng

In [6]:
# Filtering for records that include location data but no zip code
pass_1 = test_data[(test_data['ZIP CODE'].isna()) & (test_data['LOCATION'].notna())]
pass_1.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,UNIQUE KEY,VEHICLE TYPE CODE 1
8,2018-09-04,8:00,NaN,NaN,40.755370,-73.722244,"40.75537, -73.722244",GRAND CENTRAL PKWY,NaN,0.0,0.0,0,0,0,0,0,0,Following Too Closely,3973698,Sedan
15,2018-11-30,0:00,NaN,NaN,40.804993,-73.911354,"40.804993, -73.911354",BRUCKNER EXPRESSWAY,NaN,1.0,0.0,0,0,0,0,1,0,Unsafe Speed,4035528,Sedan
21,2018-03-14,11:00,NaN,NaN,40.679200,-73.955300,"40.6792, -73.9553",ATLANTIC AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inexperience,3863647,SPORT UTILITY / STATION WAGON
36,2018-06-22,8:50,NaN,NaN,40.665474,-73.867520,"40.665474, -73.86752",STANLEY AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,3926836,Sedan
40,2017-09-17,19:20,NaN,NaN,40.682410,-73.961845,"40.68241, -73.961845",FULTON STREET,NaN,0.0,0.0,0,0,0,0,0,0,Traffic Control Disregarded,3751862,SPORT UTILITY / STATION WAGON


In [7]:
# Reverse geolocation based on coordinates
url = 'https://maps.googleapis.com/maps/api/geocode/json?'

latlng = pass_1['LOCATION']

full_addresses_1 = [] # Getting full address based on coordinates


for latlng in latlng:
    query_url = f'{url}latlng={latlng}&key={google_api_key}'

    try:
        resp = requests.get(query_url)
        data_1 = resp.json()
        print(data_1)
    except:
        print('Incorrectly formatted URL') # Accounts for NaN values in 'LOCATION'
    
    try:
        full_addresses_1.append(data_1['results'][1]['formatted_address'])
    except:
        print("No Data found") # Accounts for addresses not found in the Google API
        full_addresses_1.append('NaN')

{'plus_code': {'compound_code': 'Q74H+44 New York, NY, USA', 'global_code': '87G8Q74H+44'}, 'results': [{'address_components': [{'long_name': '26121', 'short_name': '26121', 'types': ['street_number']}, {'long_name': 'Grand Central Parkway Service Road', 'short_name': 'Grand Central Parkway Service Road', 'types': ['route']}, {'long_name': 'Little Neck', 'short_name': 'Little Neck', 'types': ['neighborhood', 'political']}, {'long_name': 'Queens', 'short_name': 'Queens', 'types': ['political', 'sublocality', 'sublocality_level_1']}, {'long_name': 'Queens County', 'short_name': 'Queens County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'New York', 'short_name': 'NY', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}, {'long_name': '11362', 'short_name': '11362', 'types': ['postal_code']}], 'formatted_address': '26121 Grand Central Parkway Service Road, Little Neck, NY 1

In [8]:
full_addresses_1

['Little Neck Pkwy/Cullman Av, Queens, NY 11362, USA',
 '281 Bruckner Blvd, The Bronx, NY 10454, USA',
 '1077 Atlantic Ave, Brooklyn, NY 11238, USA',
 'Crescent St/Stanley Av, Brooklyn, NY 11208, USA',
 '22 Putnam Ave, Brooklyn, NY 11238, USA',
 '250 Bedford Park Blvd W, The Bronx, NY 10468, USA',
 'Broadway/W 133 St, New York, NY 10027, USA',
 '281 Regis Dr, Staten Island, NY 10314, USA',
 '1401 Richmond Ave, Staten Island, NY 10314, USA',
 '7501 20th Ave, Brooklyn, NY 11214, USA',
 '60-99 75th Ave, Glendale, NY 11385, USA',
 'Grand Central Pkwy, Forest Hills, NY 11375, USA',
 '48 St/55 Av, Queens, NY 11378, USA',
 '118 St Nicholas Ave, Brooklyn, NY 11237, USA',
 '60-41 Booth St, Rego Park, NY 11374, USA',
 '1444 Macombs Rd, The Bronx, NY 10452, USA',
 '142-04 Horace Harding Expy, Flushing, NY 11367, USA',
 '271 W 47th St, New York, NY 10036, USA',
 '606 Bushwick Ave, Brooklyn, NY 11206, USA',
 'E 23 St/Broadway, New York, NY 10010, USA',
 '25 Staff St, New York, NY 10034, USA',
 '232

In [9]:
pass_1['FULL_ADDRESS'] = full_addresses_1

In [10]:
pass_1.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,...,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,UNIQUE KEY,VEHICLE TYPE CODE 1,FULL_ADDRESS
8,2018-09-04,8:00,NaN,NaN,40.755370,-73.722244,"40.75537, -73.722244",GRAND CENTRAL PKWY,NaN,0.0,...,0,0,0,0,0,0,Following Too Closely,3973698,Sedan,"Little Neck Pkwy/Cullman Av, Queens, NY 11362,..."
15,2018-11-30,0:00,NaN,NaN,40.804993,-73.911354,"40.804993, -73.911354",BRUCKNER EXPRESSWAY,NaN,1.0,...,0,0,0,0,1,0,Unsafe Speed,4035528,Sedan,"281 Bruckner Blvd, The Bronx, NY 10454, USA"
21,2018-03-14,11:00,NaN,NaN,40.679200,-73.955300,"40.6792, -73.9553",ATLANTIC AVENUE,NaN,0.0,...,0,0,0,0,0,0,Driver Inexperience,3863647,SPORT UTILITY / STATION WAGON,"1077 Atlantic Ave, Brooklyn, NY 11238, USA"
36,2018-06-22,8:50,NaN,NaN,40.665474,-73.867520,"40.665474, -73.86752",STANLEY AVENUE,NaN,0.0,...,0,0,0,0,0,0,Driver Inattention/Distraction,3926836,Sedan,"Crescent St/Stanley Av, Brooklyn, NY 11208, USA"
40,2017-09-17,19:20,NaN,NaN,40.682410,-73.961845,"40.68241, -73.961845",FULTON STREET,NaN,0.0,...,0,0,0,0,0,0,Traffic Control Disregarded,3751862,SPORT UTILITY / STATION WAGON,"22 Putnam Ave, Brooklyn, NY 11238, USA"


In [11]:
# Splitting FULL_ADDRESS column to indiviual values
i = pass_1.columns.get_loc('FULL_ADDRESS')
df2 = pass_1['FULL_ADDRESS'].str.split(",", n=3, expand=True)
temp_zips = pd.concat([pass_1.iloc[:, :i], df2, pass_1.iloc[:, i+1:]], axis=1)
temp_zips.drop([0, 3], axis=1, inplace=True)
temp_zips.rename(columns={1: 'FOUND BORO'}, inplace=True)
temp_zips.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,...,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,UNIQUE KEY,VEHICLE TYPE CODE 1,FOUND BORO,2
8,2018-09-04,8:00,NaN,NaN,40.755370,-73.722244,"40.75537, -73.722244",GRAND CENTRAL PKWY,NaN,0.0,...,0,0,0,0,0,Following Too Closely,3973698,Sedan,Queens,NY 11362
15,2018-11-30,0:00,NaN,NaN,40.804993,-73.911354,"40.804993, -73.911354",BRUCKNER EXPRESSWAY,NaN,1.0,...,0,0,0,1,0,Unsafe Speed,4035528,Sedan,The Bronx,NY 10454
21,2018-03-14,11:00,NaN,NaN,40.679200,-73.955300,"40.6792, -73.9553",ATLANTIC AVENUE,NaN,0.0,...,0,0,0,0,0,Driver Inexperience,3863647,SPORT UTILITY / STATION WAGON,Brooklyn,NY 11238
36,2018-06-22,8:50,NaN,NaN,40.665474,-73.867520,"40.665474, -73.86752",STANLEY AVENUE,NaN,0.0,...,0,0,0,0,0,Driver Inattention/Distraction,3926836,Sedan,Brooklyn,NY 11208
40,2017-09-17,19:20,NaN,NaN,40.682410,-73.961845,"40.68241, -73.961845",FULTON STREET,NaN,0.0,...,0,0,0,0,0,Traffic Control Disregarded,3751862,SPORT UTILITY / STATION WAGON,Brooklyn,NY 11238


In [12]:
i = temp_zips.columns.get_loc(2)
df2 = temp_zips[2].str.split(" ", n=2, expand=True)
augmented_zips = pd.concat([temp_zips.iloc[:, :i], df2, temp_zips.iloc[:, i+1:]], axis=1)
augmented_zips.drop([0, 1], axis=1, inplace=True)
augmented_zips.rename(columns={2: 'FOUND ZIP'}, inplace=True)
augmented_zips.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,...,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,UNIQUE KEY,VEHICLE TYPE CODE 1,FOUND BORO,FOUND ZIP
8,2018-09-04,8:00,NaN,NaN,40.755370,-73.722244,"40.75537, -73.722244",GRAND CENTRAL PKWY,NaN,0.0,...,0,0,0,0,0,Following Too Closely,3973698,Sedan,Queens,11362
15,2018-11-30,0:00,NaN,NaN,40.804993,-73.911354,"40.804993, -73.911354",BRUCKNER EXPRESSWAY,NaN,1.0,...,0,0,0,1,0,Unsafe Speed,4035528,Sedan,The Bronx,10454
21,2018-03-14,11:00,NaN,NaN,40.679200,-73.955300,"40.6792, -73.9553",ATLANTIC AVENUE,NaN,0.0,...,0,0,0,0,0,Driver Inexperience,3863647,SPORT UTILITY / STATION WAGON,Brooklyn,11238
36,2018-06-22,8:50,NaN,NaN,40.665474,-73.867520,"40.665474, -73.86752",STANLEY AVENUE,NaN,0.0,...,0,0,0,0,0,Driver Inattention/Distraction,3926836,Sedan,Brooklyn,11208
40,2017-09-17,19:20,NaN,NaN,40.682410,-73.961845,"40.68241, -73.961845",FULTON STREET,NaN,0.0,...,0,0,0,0,0,Traffic Control Disregarded,3751862,SPORT UTILITY / STATION WAGON,Brooklyn,11238


In [13]:
pass_1['ZIP CODE'] = augmented_zips['FOUND ZIP']
pass_1['BOROUGH'] = augmented_zips['FOUND BORO']

In [14]:
pass_1.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,...,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,UNIQUE KEY,VEHICLE TYPE CODE 1,FULL_ADDRESS
8,2018-09-04,8:00,Queens,11362,40.755370,-73.722244,"40.75537, -73.722244",GRAND CENTRAL PKWY,NaN,0.0,...,0,0,0,0,0,0,Following Too Closely,3973698,Sedan,"Little Neck Pkwy/Cullman Av, Queens, NY 11362,..."
15,2018-11-30,0:00,The Bronx,10454,40.804993,-73.911354,"40.804993, -73.911354",BRUCKNER EXPRESSWAY,NaN,1.0,...,0,0,0,0,1,0,Unsafe Speed,4035528,Sedan,"281 Bruckner Blvd, The Bronx, NY 10454, USA"
21,2018-03-14,11:00,Brooklyn,11238,40.679200,-73.955300,"40.6792, -73.9553",ATLANTIC AVENUE,NaN,0.0,...,0,0,0,0,0,0,Driver Inexperience,3863647,SPORT UTILITY / STATION WAGON,"1077 Atlantic Ave, Brooklyn, NY 11238, USA"
36,2018-06-22,8:50,Brooklyn,11208,40.665474,-73.867520,"40.665474, -73.86752",STANLEY AVENUE,NaN,0.0,...,0,0,0,0,0,0,Driver Inattention/Distraction,3926836,Sedan,"Crescent St/Stanley Av, Brooklyn, NY 11208, USA"
40,2017-09-17,19:20,Brooklyn,11238,40.682410,-73.961845,"40.68241, -73.961845",FULTON STREET,NaN,0.0,...,0,0,0,0,0,0,Traffic Control Disregarded,3751862,SPORT UTILITY / STATION WAGON,"22 Putnam Ave, Brooklyn, NY 11238, USA"


In [15]:
pass_1.shape

(24, 21)

In [16]:
pass_1['ZIP CODE'].isna().sum()

1

In [17]:
# Using augmented values from pass_1 to main data
values_to_update = pass_1.loc[pass_1['LOCATION'].isin(test_data['LOCATION'])][['ZIP CODE', 'BOROUGH']]
values_to_update.head()

,ZIP CODE,BOROUGH
8,11362,Queens
15,10454,The Bronx
21,11238,Brooklyn
36,11208,Brooklyn
40,11238,Brooklyn


In [19]:
# Replacing missing values in test data with found zip codes and boroughs in pass 1
augmented_after_pass_1 = test_data.fillna(values_to_update)
augmented_after_pass_1.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,UNIQUE KEY,VEHICLE TYPE CODE 1
0,2017-02-12,4:15,QUEENS,11106,40.7585,-73.9249,"40.75855, -73.924866",34 AVENUE,34 STREET,0,0,0,0,0,0,0,0,Unsafe Speed,3613857,SPORT UTILITY / STATION WAGON
1,2013-07-18,12:05,BROOKLYN,11209,40.6179,-74.0293,"40.6178962, -74.0293024",92 STREET,5 AVENUE,0,0,0,0,0,0,0,0,Unspecified,143196,PASSENGER VEHICLE
2,2015-09-16,9:00,MANHATTAN,10016,40.7457,-73.9721,"40.7456822, -73.9721247",EAST 37 STREET,1 AVENUE,0,0,0,0,0,0,0,0,Other Vehicular,3297366,SPORT UTILITY / STATION WAGON
3,2013-01-19,13:15,NaN,NaN,NaN,NaN,NaN,EAST 135 STREET,LINCOLN AVENUE,0,0,0,0,0,0,0,0,Other Vehicular,73713,PASSENGER VEHICLE
4,2018-08-11,22:51,BROOKLYN,11219,40.6273,-74.01,"40.62735, -74.009964",NaN,NaN,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,3959547,Sedan
5,2013-12-23,13:30,NaN,NaN,NaN,NaN,NaN,BRUCKNER BOULEVARD,WILLIS AVENUE,0,0,0,0,0,0,0,0,Unspecified,75773,PASSENGER VEHICLE
6,2017-06-14,12:15,BRONX,10468,40.8719,-73.8882,"40.871864, -73.888214",NaN,NaN,0,0,0,0,0,0,0,0,Failure to Yield Right-of-Way,3692078,PASSENGER VEHICLE
7,2019-01-16,0:31,BROOKLYN,11220,40.6356,-74.0081,"40.63558, -74.00812",NaN,NaN,0,0,0,0,0,0,0,0,Passing Too Closely,4058081,Sedan
8,2018-09-04,8:00,Queens,11362,40.7554,-73.7222,"40.75537, -73.722244",GRAND CENTRAL PKWY,NaN,0,0,0,0,0,0,0,0,Following Too Closely,3973698,Sedan
9,2015-07-14,13:50,BROOKLYN,11201,40.6889,-73.9809,"40.6889031, -73.9809286",FLATBUSH AVENUE EXTENSION,FULTON STREET,2,0,0,0,0,0,2,0,Unspecified,3258170,PASSENGER VEHICLE


In [20]:
# Percent of zip codes missing after augmentation in pass 1
percent_missing_after_pass_1 = round(augmented_after_pass_1['ZIP CODE'].isna().sum()/len(augmented_after_pass_1['ZIP CODE']) * 100, 2)
percent_missing_after_pass_1

13.82

After pass 1 we were able to reduce the percentage of missing values from ~29% to ~14 

## 2. Reverse geocoding using Street and Cross Street data

In [44]:
# Filtering for records that include street data but no zip code and no location
pass_2 = augmented_after_pass_1[
                                (augmented_after_pass_1['ZIP CODE'].isna()) 
                                | (augmented_after_pass_1['LOCATION'].isna()) 
                                & (augmented_after_pass_1['ON STREET NAME'].notna())
                                & (augmented_after_pass_1['CROSS STREET NAME'].notna())
                            ]
pass_2.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,UNIQUE KEY,VEHICLE TYPE CODE 1
3,2013-01-19,13:15,NaN,NaN,NaN,NaN,NaN,EAST 135 STREET,LINCOLN AVENUE,0,0,0,0,0,0,0,0,Other Vehicular,73713,PASSENGER VEHICLE
5,2013-12-23,13:30,NaN,NaN,NaN,NaN,NaN,BRUCKNER BOULEVARD,WILLIS AVENUE,0,0,0,0,0,0,0,0,Unspecified,75773,PASSENGER VEHICLE
10,2014-12-21,14:30,NaN,NaN,NaN,NaN,NaN,WHITE PLAINS ROAD,PELHAM PARKWAY SOUTH,0,0,0,0,0,0,0,0,Unspecified,3141138,PASSENGER VEHICLE
11,2016-03-14,8:55,MANHATTAN,10024,NaN,NaN,NaN,CENTRAL PARK WEST,WEST 88 STREET,0,0,0,0,0,0,0,0,Passing Too Closely,3409540,PASSENGER VEHICLE
16,2013-11-19,7:55,NaN,NaN,NaN,NaN,NaN,QUEENS PLAZA SOUTH,27 STREET,0,0,0,0,0,0,0,0,Unspecified,244224,PASSENGER VEHICLE
18,2013-03-19,11:40,NaN,NaN,NaN,NaN,NaN,PELHAM PARKWAY SOUTH,STILLWELL AVENUE,0,0,0,0,0,0,0,0,Unspecified,104528,PASSENGER VEHICLE
27,2016-06-10,12:00,NaN,NaN,NaN,NaN,NaN,LONG ISLAND EXPRESSWAY,NaN,2,0,0,0,0,0,2,0,Driver Inattention/Distraction,3459940,PASSENGER VEHICLE
28,2014-11-24,17:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Prescription Medication,3126071,PASSENGER VEHICLE
29,2015-11-03,21:07,NaN,NaN,NaN,NaN,NaN,BRUCKNER BOULEVARD,BRONX RIVER AVENUE,0,0,0,0,0,0,0,0,Unspecified,3327917,PASSENGER VEHICLE
30,2013-10-03,16:00,NaN,NaN,NaN,NaN,NaN,149 AVENUE,NORTH CONDUIT AVENUE,0,0,0,0,0,0,0,0,Unspecified,231749,SPORT UTILITY / STATION WAGON


In [45]:
# Reverse geolocation based on street address
url = 'https://maps.googleapis.com/maps/api/geocode/json?'

addresses = list(zip(pass_2['ON STREET NAME'], pass_2['CROSS STREET NAME']))

full_addresses_2 = []
latitude_2 = []
longitude_2 = []

for a in addresses:
    query_url = f'{url}address={a}&key={google_api_key}'

    try:
        resp = requests.get(query_url)
        data_2 = resp.json()
        #full_data_2.append(data_2)
    except:
        print('Incorrectly formatted URL') # Accounts for NaN values in 'LOCATION'  
    
    try:
        full_addresses_2.append(data_2['results'][0]['formatted_address'])
        latitude_2.append(data_2['results'][0]['geometry']['location']['lat'])
        longitude_2.append(data_2['results'][0]['geometry']['location']['lng'])
        
    except:
        print("No Data found") # Accounts for addresses not found in the Google API
        full_addresses_2.append('NaN')
        latitude_2.append('NaN')
        longitude_2.append('NaN')

In [46]:
pass_2['FULL_ADDRESS'] = full_addresses_2
pass_2['LATITUDE'] = latitude_2
pass_2['LONGITUDE'] = longitude_2
pass_2['LOCATION'] = list(zip(latitude_2, longitude_2))
pass_2.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,...,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,UNIQUE KEY,VEHICLE TYPE CODE 1,FULL_ADDRESS
3,2013-01-19,13:15,NaN,NaN,40.809066,-73.928558,"(40.809066, -73.92855759999999)",EAST 135 STREET,LINCOLN AVENUE,0,...,0,0,0,0,0,0,Other Vehicular,73713,PASSENGER VEHICLE,"Lincoln Ave & E 135th St, The Bronx, NY 10454,..."
5,2013-12-23,13:30,NaN,NaN,40.805945,-73.925340,"(40.8059446, -73.92534030000002)",BRUCKNER BOULEVARD,WILLIS AVENUE,0,...,0,0,0,0,0,0,Unspecified,75773,PASSENGER VEHICLE,"Bruckner Blvd & Willis Ave, The Bronx, NY 1045..."
10,2014-12-21,14:30,NaN,NaN,40.855770,-73.867475,"(40.8557705, -73.8674752)",WHITE PLAINS ROAD,PELHAM PARKWAY SOUTH,0,...,0,0,0,0,0,0,Unspecified,3141138,PASSENGER VEHICLE,"2178 White Plains Rd, Pelham Parkway, NY 10462..."
11,2016-03-14,8:55,MANHATTAN,10024,40.774134,-73.977711,"(40.7741337, -73.9777111)",CENTRAL PARK WEST,WEST 88 STREET,0,...,0,0,0,0,0,0,Passing Too Closely,3409540,PASSENGER VEHICLE,"88 Central Park West, New York, NY 10023, USA"
16,2013-11-19,7:55,NaN,NaN,40.750185,-73.939855,"(40.7501851, -73.9398551)",QUEENS PLAZA SOUTH,27 STREET,0,...,0,0,0,0,0,0,Unspecified,244224,PASSENGER VEHICLE,"27th St & Queens Plaza S, Queens, NY 11101, USA"
18,2013-03-19,11:40,NaN,NaN,40.856477,-73.838082,"(40.8564775, -73.83808189999999)",PELHAM PARKWAY SOUTH,STILLWELL AVENUE,0,...,0,0,0,0,0,0,Unspecified,104528,PASSENGER VEHICLE,"Stillwell Ave & Pelham Pkwy S, The Bronx, NY 1..."
27,2016-06-10,12:00,NaN,NaN,40.800976,-73.334196,"(40.8009757, -73.3341958)",LONG ISLAND EXPRESSWAY,NaN,2,...,0,0,0,0,2,0,Driver Inattention/Distraction,3459940,PASSENGER VEHICLE,"I-495, New York, USA"
28,2014-11-24,17:00,NaN,NaN,36.054574,-95.923037,"(36.0545737, -95.923037)",NaN,NaN,0,...,0,0,0,0,0,0,Prescription Medication,3126071,PASSENGER VEHICLE,"7502 S Winston Pl, Tulsa, OK 74136, USA"
29,2015-11-03,21:07,NaN,NaN,40.822715,-73.882510,"(40.8227154, -73.8825098)",BRUCKNER BOULEVARD,BRONX RIVER AVENUE,0,...,0,0,0,0,0,0,Unspecified,3327917,PASSENGER VEHICLE,"Bronx River Av/Bruckner Blvd, The Bronx, NY 10..."
30,2013-10-03,16:00,NaN,NaN,40.667108,-73.812489,"(40.6671078, -73.8124888)",149 AVENUE,NORTH CONDUIT AVENUE,0,...,0,0,0,0,0,0,Unspecified,231749,SPORT UTILITY / STATION WAGON,"149th Ave & N Conduit Ave, Queens, NY 11420, USA"


In [47]:
# Splitting FULL_ADDRESS column to indiviual values
i = pass_2.columns.get_loc('FULL_ADDRESS')
df2 = pass_2['FULL_ADDRESS'].str.split(",", n=3, expand=True)
temp_zips = pd.concat([pass_2.iloc[:, :i], df2, pass_2.iloc[:, i+1:]], axis=1)
temp_zips.drop([0, 3], axis=1, inplace=True)
temp_zips.rename(columns={1: 'FOUND BORO'}, inplace=True)
temp_zips.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,...,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,UNIQUE KEY,VEHICLE TYPE CODE 1,FOUND BORO,2
3,2013-01-19,13:15,NaN,NaN,40.809066,-73.928558,"(40.809066, -73.92855759999999)",EAST 135 STREET,LINCOLN AVENUE,0,...,0,0,0,0,0,Other Vehicular,73713,PASSENGER VEHICLE,The Bronx,NY 10454
5,2013-12-23,13:30,NaN,NaN,40.805945,-73.925340,"(40.8059446, -73.92534030000002)",BRUCKNER BOULEVARD,WILLIS AVENUE,0,...,0,0,0,0,0,Unspecified,75773,PASSENGER VEHICLE,The Bronx,NY 10454
10,2014-12-21,14:30,NaN,NaN,40.855770,-73.867475,"(40.8557705, -73.8674752)",WHITE PLAINS ROAD,PELHAM PARKWAY SOUTH,0,...,0,0,0,0,0,Unspecified,3141138,PASSENGER VEHICLE,Pelham Parkway,NY 10462
11,2016-03-14,8:55,MANHATTAN,10024,40.774134,-73.977711,"(40.7741337, -73.9777111)",CENTRAL PARK WEST,WEST 88 STREET,0,...,0,0,0,0,0,Passing Too Closely,3409540,PASSENGER VEHICLE,New York,NY 10023
16,2013-11-19,7:55,NaN,NaN,40.750185,-73.939855,"(40.7501851, -73.9398551)",QUEENS PLAZA SOUTH,27 STREET,0,...,0,0,0,0,0,Unspecified,244224,PASSENGER VEHICLE,Queens,NY 11101


In [48]:
i = temp_zips.columns.get_loc(2)
df2 = temp_zips[2].str.split(" ", n=2, expand=True)
augmented_zips = pd.concat([temp_zips.iloc[:, :i], df2, temp_zips.iloc[:, i+1:]], axis=1)
augmented_zips.drop([0, 1], axis=1, inplace=True)
augmented_zips.rename(columns={2: 'FOUND ZIP'}, inplace=True)
augmented_zips.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,...,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,UNIQUE KEY,VEHICLE TYPE CODE 1,FOUND BORO,FOUND ZIP
3,2013-01-19,13:15,NaN,NaN,40.809066,-73.928558,"(40.809066, -73.92855759999999)",EAST 135 STREET,LINCOLN AVENUE,0,...,0,0,0,0,0,Other Vehicular,73713,PASSENGER VEHICLE,The Bronx,10454
5,2013-12-23,13:30,NaN,NaN,40.805945,-73.925340,"(40.8059446, -73.92534030000002)",BRUCKNER BOULEVARD,WILLIS AVENUE,0,...,0,0,0,0,0,Unspecified,75773,PASSENGER VEHICLE,The Bronx,10454
10,2014-12-21,14:30,NaN,NaN,40.855770,-73.867475,"(40.8557705, -73.8674752)",WHITE PLAINS ROAD,PELHAM PARKWAY SOUTH,0,...,0,0,0,0,0,Unspecified,3141138,PASSENGER VEHICLE,Pelham Parkway,10462
11,2016-03-14,8:55,MANHATTAN,10024,40.774134,-73.977711,"(40.7741337, -73.9777111)",CENTRAL PARK WEST,WEST 88 STREET,0,...,0,0,0,0,0,Passing Too Closely,3409540,PASSENGER VEHICLE,New York,10023
16,2013-11-19,7:55,NaN,NaN,40.750185,-73.939855,"(40.7501851, -73.9398551)",QUEENS PLAZA SOUTH,27 STREET,0,...,0,0,0,0,0,Unspecified,244224,PASSENGER VEHICLE,Queens,11101


In [49]:
pass_2['ZIP CODE'] = augmented_zips['FOUND ZIP']
pass_2['BOROUGH'] = augmented_zips['FOUND BORO']

In [50]:
pass_2.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,...,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,UNIQUE KEY,VEHICLE TYPE CODE 1,FULL_ADDRESS
3,2013-01-19,13:15,The Bronx,10454,40.809066,-73.928558,"(40.809066, -73.92855759999999)",EAST 135 STREET,LINCOLN AVENUE,0,...,0,0,0,0,0,0,Other Vehicular,73713,PASSENGER VEHICLE,"Lincoln Ave & E 135th St, The Bronx, NY 10454,..."
5,2013-12-23,13:30,The Bronx,10454,40.805945,-73.925340,"(40.8059446, -73.92534030000002)",BRUCKNER BOULEVARD,WILLIS AVENUE,0,...,0,0,0,0,0,0,Unspecified,75773,PASSENGER VEHICLE,"Bruckner Blvd & Willis Ave, The Bronx, NY 1045..."
10,2014-12-21,14:30,Pelham Parkway,10462,40.855770,-73.867475,"(40.8557705, -73.8674752)",WHITE PLAINS ROAD,PELHAM PARKWAY SOUTH,0,...,0,0,0,0,0,0,Unspecified,3141138,PASSENGER VEHICLE,"2178 White Plains Rd, Pelham Parkway, NY 10462..."
11,2016-03-14,8:55,New York,10023,40.774134,-73.977711,"(40.7741337, -73.9777111)",CENTRAL PARK WEST,WEST 88 STREET,0,...,0,0,0,0,0,0,Passing Too Closely,3409540,PASSENGER VEHICLE,"88 Central Park West, New York, NY 10023, USA"
16,2013-11-19,7:55,Queens,11101,40.750185,-73.939855,"(40.7501851, -73.9398551)",QUEENS PLAZA SOUTH,27 STREET,0,...,0,0,0,0,0,0,Unspecified,244224,PASSENGER VEHICLE,"27th St & Queens Plaza S, Queens, NY 11101, USA"


In [51]:
pass_2.shape

(25, 21)

In [52]:
pass_2['ZIP CODE'].isna().sum()

5

In [53]:
# Using augmented values from pass_1 to main data using date to match
values_to_update = pass_2.loc[pass_2['UNIQUE KEY'].isin(augmented_after_pass_1['UNIQUE KEY'])][['ZIP CODE', 'BOROUGH', 'LATITUDE', 'LONGITUDE', 'LOCATION']]
values_to_update.head()

,ZIP CODE,BOROUGH,LATITUDE,LONGITUDE,LOCATION
3,10454,The Bronx,40.809066,-73.928558,"(40.809066, -73.92855759999999)"
5,10454,The Bronx,40.805945,-73.925340,"(40.8059446, -73.92534030000002)"
10,10462,Pelham Parkway,40.855770,-73.867475,"(40.8557705, -73.8674752)"
11,10023,New York,40.774134,-73.977711,"(40.7741337, -73.9777111)"
16,11101,Queens,40.750185,-73.939855,"(40.7501851, -73.9398551)"


In [54]:
# Replacing missing values in test data with found zip codes and boroughs in pass 1
augmented_after_pass_2 = augmented_after_pass_1.fillna(values_to_update)
augmented_after_pass_2.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,UNIQUE KEY,VEHICLE TYPE CODE 1
0,2017-02-12,4:15,QUEENS,11106,40.7585,-73.9249,"40.75855, -73.924866",34 AVENUE,34 STREET,0,0,0,0,0,0,0,0,Unsafe Speed,3613857,SPORT UTILITY / STATION WAGON
1,2013-07-18,12:05,BROOKLYN,11209,40.6179,-74.0293,"40.6178962, -74.0293024",92 STREET,5 AVENUE,0,0,0,0,0,0,0,0,Unspecified,143196,PASSENGER VEHICLE
2,2015-09-16,9:00,MANHATTAN,10016,40.7457,-73.9721,"40.7456822, -73.9721247",EAST 37 STREET,1 AVENUE,0,0,0,0,0,0,0,0,Other Vehicular,3297366,SPORT UTILITY / STATION WAGON
3,2013-01-19,13:15,The Bronx,10454,40.8091,-73.9286,"(40.809066, -73.92855759999999)",EAST 135 STREET,LINCOLN AVENUE,0,0,0,0,0,0,0,0,Other Vehicular,73713,PASSENGER VEHICLE
4,2018-08-11,22:51,BROOKLYN,11219,40.6273,-74.01,"40.62735, -74.009964",NaN,NaN,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,3959547,Sedan


In [55]:
# Percent of zip codes missing after augmentation in pass 2
percent_missing_after_pass_2 = round(augmented_after_pass_2['ZIP CODE'].isna().sum()/len(augmented_after_pass_2['ZIP CODE']) * 100, 2)
percent_missing_after_pass_2

2.63

In [57]:
augmented_after_pass_2.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,UNIQUE KEY,VEHICLE TYPE CODE 1
0,2017-02-12,4:15,QUEENS,11106,40.7585,-73.9249,"40.75855, -73.924866",34 AVENUE,34 STREET,0,0,0,0,0,0,0,0,Unsafe Speed,3613857,SPORT UTILITY / STATION WAGON
1,2013-07-18,12:05,BROOKLYN,11209,40.6179,-74.0293,"40.6178962, -74.0293024",92 STREET,5 AVENUE,0,0,0,0,0,0,0,0,Unspecified,143196,PASSENGER VEHICLE
2,2015-09-16,9:00,MANHATTAN,10016,40.7457,-73.9721,"40.7456822, -73.9721247",EAST 37 STREET,1 AVENUE,0,0,0,0,0,0,0,0,Other Vehicular,3297366,SPORT UTILITY / STATION WAGON
3,2013-01-19,13:15,The Bronx,10454,40.8091,-73.9286,"(40.809066, -73.92855759999999)",EAST 135 STREET,LINCOLN AVENUE,0,0,0,0,0,0,0,0,Other Vehicular,73713,PASSENGER VEHICLE
4,2018-08-11,22:51,BROOKLYN,11219,40.6273,-74.01,"40.62735, -74.009964",NaN,NaN,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,3959547,Sedan


In [59]:
augmented_after_pass_2[augmented_after_pass_2['ZIP CODE'].isna()]

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,UNIQUE KEY,VEHICLE TYPE CODE 1
27,2016-06-10,12:00,New York,None,40.801,-73.3342,"(40.8009757, -73.3341958)",LONG ISLAND EXPRESSWAY,NaN,2,0,0,0,0,0,2,0,Driver Inattention/Distraction,3459940,PASSENGER VEHICLE
32,2017-11-18,13:00,Queens,None,40.6972,-73.7275,"(40.6971555, -73.7275489)",CROSS ISLAND PARKWAY,NaN,0,0,0,0,0,0,0,0,Following Too Closely,3792415,PASSENGER VEHICLE
66,2012-07-26,7:05,The Bronx,None,40.8328,-73.9051,"(40.8327501, -73.9050863)",THIRD AVENUE,CROSS BRONX EXPRESSWAY,1,0,0,0,0,0,1,0,Unspecified,100014,LARGE COM VEH(6 OR MORE TIRES)
128,2015-12-23,18:30,New York,None,40.7376,-73.8365,"(40.7376426, -73.8365025)",20 AVENUE,WHITESTONE EXPRESSWAY E SVC RD,0,0,0,0,0,0,0,0,NaN,3359768,PASSENGER VEHICLE


In [60]:
# Exporting csv
augmented_after_pass_2.to_csv('Resources/augmented_test_data.csv', index=False)

## Summary

**Using the Google Geocode API I was able to find all the values in out test dataset, through two passes:**
- First pass used latitude and longitude coordinates
- Second pass used street address and cross street


Using these two pieces of information, I was able to locate the exact address for all the records fill in the missing values. The total missing values decreased from ~29% to ~2.6% in the test dataset

4 records do not have zip codes. These incidents occured on highways and expressways, which might explain why they're missing.

